# Recurrent postprocessing: v1
This file takes as input a sequence of data from the CNN, and attempts to refine that into a more accurate output command.

Some credit belongs to https://github.com/harvitronix/five-video-classification-methods/blob/master/models.py for providing inspiration.

In [1]:
#Create references to important directories we will use over and over
import os, sys
DATA_HOME_DIR = '/home/nathan/olin/spring2017/line-follower/line-follower/data'

In [2]:
#import modules
import numpy as np
from glob import glob
from PIL import Image
from tqdm import tqdm
import bcolz

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from keras.layers import *
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.wrappers import TimeDistributed
from keras.metrics import categorical_crossentropy, categorical_accuracy

Using TensorFlow backend.


In [4]:
%cd $DATA_HOME_DIR

path = DATA_HOME_DIR
train_path1=path + '/sun_apr_16_office_full_line_1'
train_path2=path + '/qea_blob_1'
# valid_path1=path + '/sun_apr_16_office_full_line_2'
# valid_path2=path + '/qea_blob_2'
valid_path1=path + '/qea-square_3'#+ '/sun_apr_16_office_full_line_2'

# train_path=path + '/qea_blob_1'
# valid_path=path + '/qea_blob_2'

/home/nathan/olin/spring2017/line-follower/line-follower/data


## Gather data

In [5]:
INPUT_LEN = 512 # The number of columns in the CSV
WINDOW_SIZE = 16

In [6]:
def load_array(fname):
    return bcolz.open(fname)[:]

def windows(X, Y, seq_len=10):
    assert len(X) == len(Y)
    
    result = []
    for index in range(X.shape[0] - seq_len):
        result.append([X[index:index+seq_len], Y[index+seq_len-1]])
    result = np.array(result)
#     np.random.shuffle(result)
    return np.array(list(result[:,0])), np.array(list(result[:,1]))

def get_data(paths):
    Y_return = []
    for path in paths:
        %cd $path
        Y_train = np.genfromtxt('cmd_vel.csv', delimiter=',')[:,1] # only use turning angle
        Y_train = np.concatenate((Y_train, Y_train*-1))
        
        Y_return.extend(Y_train)
        
    X_all = load_array(paths[-1]+'/X_train_features3.b')
    X_all = np.reshape(X_all, (len(X_all), INPUT_LEN))
        
    print (len(X_all), len(Y_return))
    
    X_windowed, Y_windowed = windows(X_all, Y_return, WINDOW_SIZE)

    return np.array(X_windowed), np.array(Y_windowed)

In [7]:
X_train, Y_train = get_data([train_path1, train_path2])
X_valid, Y_valid = get_data([valid_path1])

/home/nathan/olin/spring2017/line-follower/line-follower/data/sun_apr_16_office_full_line_1
/home/nathan/olin/spring2017/line-follower/line-follower/data/qea_blob_1
1312 1312
/home/nathan/olin/spring2017/line-follower/line-follower/data/qea-square_3
190 190


In [8]:
X_train.shape

(1296, 16, 512)

## Network

In [9]:
in_shape = (WINDOW_SIZE, INPUT_LEN)

In [10]:
def get_model():
    model = Sequential([
            LSTM(2048, return_sequences=False, input_shape=in_shape),
            Dropout(0.5),
#             Flatten(input_shape=in_shape),
            Dense(512, activation='relu'),
#             Dense(512, activation='relu'),
#             Dense(512, activation='relu'),
            Dropout(0.5),
            Dense(1)
        ])
    model.compile(loss='mean_absolute_error', optimizer='adam')
    
    return model
    
model = get_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 2048)          20979712    lstm_input_1[0][0]               
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 2048)          0           lstm_1[0][0]                     
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 512)           1049088     dropout_1[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 512)           0           dense_1[0][0]                    
___________________________________________________________________________________________

### Train the model

In [12]:
%cd $DATA_HOME_DIR
model.load_weights('LSTM_postprocessor_v1.h5')

/home/nathan/olin/spring2017/line-follower/line-follower/data


In [32]:
history = model.fit(X_train, Y_train,#X_train[:,-1],
                    batch_size = 96,
                    nb_epoch=150,
                    validation_data=(X_valid, Y_valid),
                    verbose=True)

Train on 1296 samples, validate on 578 samples
Epoch 1/150
1296/1296 [==============================] - 1s - loss: 0.0138 - val_loss: 0.0398
Epoch 2/150
1296/1296 [==============================] - 1s - loss: 0.0138 - val_loss: 0.0398
Epoch 3/150
1296/1296 [==============================] - 1s - loss: 0.0138 - val_loss: 0.0391
Epoch 4/150
1296/1296 [==============================] - 1s - loss: 0.0134 - val_loss: 0.0400
Epoch 5/150
1296/1296 [==============================] - 1s - loss: 0.0136 - val_loss: 0.0393
Epoch 6/150
1296/1296 [==============================] - 1s - loss: 0.0127 - val_loss: 0.0390
Epoch 7/150
1296/1296 [==============================] - 1s - loss: 0.0122 - val_loss: 0.0393
Epoch 8/150
1296/1296 [==============================] - 1s - loss: 0.0123 - val_loss: 0.0391
Epoch 9/150
1296/1296 [==============================] - 1s - loss: 0.0122 - val_loss: 0.0385
Epoch 10/150
1296/1296 [==============================] - 1s - loss: 0.0117 - val_loss: 0.0393
Epoch 11/150

In [13]:
conv_predictions = X_train[:,-1]
recurrent_predictions = model.predict(X_train)
ground_truth = Y_train
for x,y,z in zip(conv_predictions[:,0], ground_truth, recurrent_predictions[:,0]):
    print ("{:07f}\t{:07f}\t{:07f}\t".format(x,y,z))

0.000000	0.009191	-0.001004	
0.000000	-0.000000	-0.000995	
0.000000	-0.000000	-0.000988	
0.000000	-0.000000	-0.000984	
0.000000	-0.000000	-0.000985	
0.000000	-0.000000	-0.000986	
0.000000	-0.084054	-0.092041	
0.000000	-0.000000	-0.001812	
0.000000	-0.000000	-0.001331	
0.000000	-0.000000	-0.001100	
0.000000	-0.000000	-0.001096	
0.000000	-0.000000	-0.001089	
0.000000	-0.000000	-0.001059	
0.000000	0.064105	0.063707	
0.000000	0.036648	0.032237	
0.000000	0.006695	-0.000578	
0.000000	0.046632	0.044549	
0.000000	0.076585	0.073129	
0.000000	-0.000000	-0.000434	
0.000000	-0.000000	-0.000453	
0.000000	0.031656	0.035124	
0.000000	0.061609	0.038705	
0.000000	0.096564	0.094046	
0.000000	0.049128	0.046650	
0.000000	0.026663	0.027809	
0.000000	0.153974	0.090551	
0.000000	-0.000000	-0.000555	
0.000000	0.069097	0.068128	
0.000000	0.079081	0.081587	
0.000000	0.054120	0.055685	
0.000000	0.049128	0.047165	
0.000000	0.049128	0.046470	
0.000000	0.079081	0.078815	
0.000000	0.081577	0.080389	
0.000000	0.02666

In [14]:
conv_predictions = X_valid[:,-1]
recurrent_predictions = model.predict(X_valid)
ground_truth = Y_valid
for x,y,z in zip(conv_predictions[:,0], ground_truth, recurrent_predictions[:,0]):
    print ("{:07f}\t{:07f}\t{:07f}\t".format(x,y,z))

0.000000	0.193911	0.081758	
0.000000	0.173942	0.076677	
0.000000	0.119028	0.014650	
0.000000	0.084074	-0.000172	
0.000000	0.014183	-0.004128	
0.000000	-0.000000	-0.088243	
0.000000	0.029159	-0.082580	
0.000000	0.001702	-0.083135	
0.000000	-0.000000	-0.002743	
0.000000	-0.000000	-0.002158	
0.000000	-0.000000	-0.080231	
0.000000	-0.000000	0.173954	
0.000000	-0.000000	0.167893	
0.000000	-0.000000	-0.098979	
0.000000	0.046632	-0.107860	
0.000000	0.300000	-0.001017	
0.000000	0.300000	-0.036726	
0.000000	0.300000	-0.039343	
0.000000	0.300000	-0.060210	
0.000000	0.300000	-0.001295	
0.000000	0.300000	0.066367	
0.000000	-0.000000	-0.001154	
0.000000	-0.000000	-0.002765	
0.000000	-0.099040	-0.063886	
0.000000	-0.044117	-0.055263	
0.000000	-0.000000	-0.001552	
0.000000	-0.000000	-0.001260	
0.000000	-0.000000	-0.001398	
0.000000	-0.000000	0.003892	
0.000000	-0.000000	0.081466	
0.000000	-0.000000	0.000975	
0.000000	0.104052	0.007476	
0.000000	0.096564	-0.082542	
0.000000	0.034152	-0.004605	
0.00000

In [ ]:
plt.plot(ground_truth[:100])

## Analyze training

In [ ]:
sns.tsplot(history.history['val_loss'])

In [ ]:
for x,y in zip(X_valid, Y_valid):
    print ("{}\t{}".format(np.reshape(x,(10,)),y))